In [55]:
%run base.ipynb

In [56]:
from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

# End-2-End-Test
Als QA-Lab-Team möchten wir einen einfachen E2E-Test aufsetzen, der die Gesamtfunktionalität unseres Prototypen testet

# 1. Erstellen der Testdaten von Dataland
Erstellen von  10 Nuclear & Gas Datasets, die wir in unserem Testdurchlauf verwenden möchten

In [57]:
from pathlib import Path

from dataland_qa_lab.dataland.provide_test_data import provide_test_data

project_root = Path().resolve().parent  # noqa: FURB177
pdf_path = project_root / "data" / "pdfs"
json_path = project_root / "data" / "jsons"
test_data = provide_test_data(pdf_path=pdf_path, json_path=json_path)

Data-IDs der 10 Testdatensätze

In [58]:
print(test_data)

['e7e65724-bff1-4de4-98df-d53023b3cd22', '1351a8af-95e2-477b-9faf-929979ed77fa', 'c0858eb7-febe-4c73-939d-ac9ffcfd3fd9', '00ff5cf4-77b3-439f-beec-2eaafac91102', 'dafbeec9-2b78-49ad-b7bf-52f5cdf6f90c', '40427de9-7234-43a2-8e50-9343f088b539', '3018f74f-0299-4024-9ff1-be8e0f226ba1', '692d1751-3d1b-447c-b68a-1e9b73aac6fa', 'd27bbed9-43c9-49f4-9ce0-9b63fc30848f', 'ce7e9e58-66a8-47fb-9831-6d2c2515888b']


Company-Ids der 10 Testdatensätze

In [59]:
from dataland_qa_lab.dataland import provide_test_data

companies = ["concordia", "covestro", "deka", "enbw", "enel", "eon", "iberdrola", "munichre", "rwe", "total"]

company_ids = []

for company in companies:
    company_id = provide_test_data.get_company_id(company)
    company_ids.append(company_id)
    print(company_id)



7af5fe86-8a96-4459-9550-48c4426a8e58
a5ab3478-f352-469a-a944-252595fc7fc5
dfbbcff4-ba4d-46ee-ad53-4a4a91159a77
36ea3185-d665-41b7-a14f-4f50fa61b6db
3ef3eedd-59b1-46a5-aaa9-25d0d3363caf
8863987f-df05-452f-8067-11f72e771345
84b1b7dc-2c14-45b8-af25-0b271764416a
7110b806-efe3-42b9-9607-f3b91d8aac18
b6e5afb5-38cc-4cb2-a25d-f3acc6a3ce93
73a68944-c272-416d-b199-cd4a30579c00


Basierend auf der CompanyID laden wir nun das zugehörige Nuclear & Gas Dataset und frsgen den ersten Datenpunkt aus Meldebogen 1 ab

In [60]:
import dataland_qa_lab.dataland.get_data as qa

year = "2023"
all_values_426 = []
all_datasource_reference_bytes = []
all_datasets = []

for i in company_ids:

    # Laden aller Datasets aus jeder Periode
    all_data = qa.get_all_company_datasets(company_id=i)

    # Data_ID der gewünschten Periode des Datensets erhalten
    data_id = qa.get_data_id_by_year(company_id=i, year=year)
    print("Data_ID = " + data_id)

    # Laden eines bestimmten Datensets einer bestimmten Periode
    all_datasets.append(qa.get_dataset_by_year(company_id=i, year=year))

    # Rückgabe des ersten Wertes in dem bestimmten Datenset
    all_values_426.append(qa.get_value1_by_year(company_id=i, year=year))
    print("Value 1 = " + qa.get_value1_by_year(company_id=i, year=year))

    # Erhalten der Datenreferenz in Bytes, wenn eine Datenquelle hinterlegt ist
    all_datasource_reference_bytes.append(qa.get_datasource_reference_bytes(company_id=i, year=year))

Data_ID = e7e65724-bff1-4de4-98df-d53023b3cd22
Value 1 = No
Data_ID = 1351a8af-95e2-477b-9faf-929979ed77fa
Value 1 = No
Data_ID = c0858eb7-febe-4c73-939d-ac9ffcfd3fd9
Value 1 = No
Data_ID = 00ff5cf4-77b3-439f-beec-2eaafac91102
Value 1 = No
Data_ID = dafbeec9-2b78-49ad-b7bf-52f5cdf6f90c
Value 1 = No
Data_ID = 40427de9-7234-43a2-8e50-9343f088b539
Value 1 = No
Data_ID = 3018f74f-0299-4024-9ff1-be8e0f226ba1
Value 1 = No
Data_ID = 692d1751-3d1b-447c-b68a-1e9b73aac6fa
Value 1 = No
Data_ID = d27bbed9-43c9-49f4-9ce0-9b63fc30848f
Value 1 = No
Data_ID = ce7e9e58-66a8-47fb-9831-6d2c2515888b
Value 1 = No


In [ ]:
for o in all_datasets:
    print(o)
    print("\n")

# 2. Laden der Datenquelle von Dataland und Überführen in Textform

Finden des relevanten Bereichs im PDF-Dokument

In [61]:
import io

import pypdf  # type: ignore

all_page_number = []
all_file_id = []
all_file_name = []
all_tag_name = []
all_page = []

for t in all_datasets:

    dataset_section426 = t.data.general.general.nuclear_energy_related_activities_section426
    all_page_number.append(dataset_section426.data_source.page)

    all_file_id.append(dataset_section426.data_source.file_reference)
    all_file_name.append(dataset_section426.data_source.file_name)
    all_tag_name.append(dataset_section426.data_source.tag_name)
    all_page.append(dataset_section426.data_source.page)

    pdf = dataland_client.documents_api.get_document(dataset_section426.data_source.file_reference)
    pdf_stream = io.BytesIO(pdf)
    pdf_reader = pypdf.PdfReader(pdf_stream)

In [64]:
for u in all_page_number:
    print(u)

57
200
116
152
232
294
295
65
104
317


In [65]:
import dataland_qa_lab.dataland.data_extraction as qa_lab

all_values_426_pdf = []

i = 0
for z in all_datasets:  # noqa: B007

    relevant_page_number = int(all_page_number[i])

    relevant_page = qa_lab.get_relevant_page_of_pdf(relevant_page_number, pdf_reader)

    text_of_page = qa_lab.extraxt_text_of_pdf(relevant_page)

    section_426 = qa_lab.extract_section_426(text_of_page)
    all_values_426_pdf.append(section_426)

    print(section_426)

    i += 1

No
No
No
No
No
No
No
No
No
No


# 4. Vergleich der beiden Werte

In [66]:
for r in range(len(all_values_426)):
    print(str(r + 1) + "." + " Wert in Dataland:" + "\t" + all_values_426[r])
    print(str(r + 1) + "." " Wert im Bericht:" + "\t" + all_values_426_pdf[r])

1. Wert in Dataland:	No
1. Wert im Bericht:	No
2. Wert in Dataland:	No
2. Wert im Bericht:	No
3. Wert in Dataland:	No
3. Wert im Bericht:	No
4. Wert in Dataland:	No
4. Wert im Bericht:	No
5. Wert in Dataland:	No
5. Wert im Bericht:	No
6. Wert in Dataland:	No
6. Wert im Bericht:	No
7. Wert in Dataland:	No
7. Wert im Bericht:	No
8. Wert in Dataland:	No
8. Wert im Bericht:	No
9. Wert in Dataland:	No
9. Wert im Bericht:	No
10. Wert in Dataland:	No
10. Wert im Bericht:	No


Verdict mithilfe von if-statements festlegen

In [67]:
from dataland_qa.models.qa_report_data_point_verdict import QaReportDataPointVerdict

all_verdicts = []

for e in range(len(all_values_426)):
    if all_values_426[e] == all_values_426_pdf[e]:
        all_verdicts.append(QaReportDataPointVerdict.QAACCEPTED)
    else:
        all_verdicts.append(QaReportDataPointVerdict.QAREJECTED)

for f in range(len(all_verdicts)):
    print(all_verdicts[f])

QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED


# 5. Erstellen und Abschicken eines QA Report

In [68]:
from dataland_qa_lab.dataland import send_report_to_dataland as report

for i in range(len(all_values_426)):

    company_report_data = report.CompanyReportData(file_reference=all_file_id[i], file_name=all_file_name[i], publication_date=None)  # noqa: E501
    document_reference_data = report.DocumentReferenceData(file_reference=all_file_id[i], page="8", tag_name="Test", file_name=all_file_name[i])  # noqa: E501

    report_data_instance = report.ReportData(
        commentqareportdatapointmapstringcompanyreport="Kommentar zur QA-Zuordnung",
        commentqareportdatapointextendeddatapointyesno="Ja",
        commentextendeddatapointyesno="Nein",
        verdictqareportdatapointmapstringcompanyreport=all_verdicts[i],
        verdictqareportdatapointextendeddatapointyesno=all_verdicts[i],
        corrected_data={
            "1.": company_report_data
        },
        value=section_426,
        quality="Reported",
        data_source=document_reference_data
    )

    report.send_report_to_dataland_method(data_id=data_id, report_data=report_data_instance, dataland_client=dataland_client)  # noqa: E501
    print(report_data_instance)

commentqareportdatapointmapstringcompanyreport='Kommentar zur QA-Zuordnung' commentqareportdatapointextendeddatapointyesno='Ja' commentextendeddatapointyesno='Nein' verdictqareportdatapointmapstringcompanyreport=<QaReportDataPointVerdict.QAACCEPTED: 'QaAccepted'> verdictqareportdatapointextendeddatapointyesno=<QaReportDataPointVerdict.QAACCEPTED: 'QaAccepted'> corrected_data={'1.': CompanyReportData(file_reference='0a8eebb9e32d3c0a32a1083699352018afcbbe39458ab8441cd0c8985a466a59', file_name='2023_DNK_Concordia_Versicherungen', publication_date=None)} value='No' quality='Reported' data_source=DocumentReferenceData(file_reference='0a8eebb9e32d3c0a32a1083699352018afcbbe39458ab8441cd0c8985a466a59', page='8', tag_name='Test', file_name='2023_DNK_Concordia_Versicherungen')
commentqareportdatapointmapstringcompanyreport='Kommentar zur QA-Zuordnung' commentqareportdatapointextendeddatapointyesno='Ja' commentextendeddatapointyesno='Nein' verdictqareportdatapointmapstringcompanyreport=<QaReportD